# Getting Started - High-level API

- Ladder offers a high-level API for running models in settings where there are multiple conditions.

- It is very easy to get started, as long as you have your AnnData ready! See below for starting out with Patches.

In [ ]:
import anndata as ad
import scanpy as sc

from ladder.data import get_data
from ladder.scripts import CrossConditionWorkflow, InterpretableWorkflow # Workflow APIs, read more on which one you'd like for your case!

In [ ]:
# Download the data object - skip if you have your data
get_data("Vu")

In [ ]:
# Load the anndata object
anndata = ad.read_h5ad("data/vu_2022_ay_wh.h5ad")

## Do optional preprocessing
## IMPORTANT : Model input (anndata.X should be raw counts!!)
anndata.layers["normalized"] = anndata.X

# Find/subset HVGs & swap to raw counts
sc.pp.highly_variable_genes(anndata, n_top_genes=3000, batch_key="sample")
sc.pl.highly_variable_genes(anndata)

anndata = anndata[:, anndata.var["highly_variable"]]
anndata.X = anndata.layers["counts"]

In [ ]:
# Initialize workflow object
workflow = InterpretableWorkflow(anndata, verbose=True, random_seed=42)

# Define the condition classes & batch key to prepare the data
factors = ["time", "age", "broad_type"]
workflow.prep_model(factors, batch_key="sample", model_type='Patches', model_args={'ld_normalize' : True})

# Train the model
workflow.run_model(max_epochs=2000, convergence_threshold=1e-5, convergence_window=2000) # Lower the convergence threshold & increase the window if you need a more accurate model, will increase training time